In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spoiler-non-context/test_sentences.csv
/kaggle/input/spoiler-non-context/train_sentences_ou.csv


In [2]:
import wandb
wandb.login(key ='b69b147158077cff1cdc737669e9cf8c9d35bfa6')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:


import pandas as pd
import torch
from transformers import AutoTokenizer
import numpy as np
from transformers import AutoConfig, AutoModelForSequenceClassification
from transformers import CanineForSequenceClassification, CanineTokenizer, BertTokenizer
from torch.utils.data import Dataset
from datasets import load_metric
from sklearn.metrics import roc_auc_score
from scipy.special import softmax
from transformers import TrainingArguments
from transformers import Trainer
from sklearn.utils import resample
import torch.nn as nn
from datasets import load_dataset

from sklearn.model_selection import train_test_split

In [4]:
def compute_metrics(eval_pred):
    precision = load_metric("precision")
    recall = load_metric("recall")
    f1 = load_metric("f1")
    acc = load_metric("accuracy")
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision = precision.compute(predictions=predictions, average = "macro", references=labels)["precision"]
    recall = recall.compute(predictions=predictions, average = "macro", references=labels)["recall"]
    f1 = f1.compute(predictions=predictions, average = "macro", references=labels)["f1"]
    acc = acc.compute(predictions=predictions, references=labels)["accuracy"]
    # check
    auc = roc_auc_score(labels, softmax(logits, axis=1)[:, 1])
    return {"precision": precision, "recall": recall, "acc": acc, "f1": f1 , 'auc':auc}

In [5]:
class SpoilerDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [6]:
class RandomTransformerClassifier:
    def __init__(self, trainingset, validationset, testset, num_classes, epochs=10, batch_size=8, model_name='distilbert-base-uncased', max_sequence_length=2048):
        self.trainingset = trainingset
        self.validationset = validationset
        self.testset = testset
        self.num_classes = num_classes
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
            
       
        self.base_model = AutoModelForSequenceClassification.from_pretrained(self.model_name, num_labels=num_classes)
        self.training_args = TrainingArguments(
        output_dir='./results',          # output directory
        #do_train=True,
        do_eval=True,
        evaluation_strategy='epoch',
        save_strategy='epoch',
        load_best_model_at_end=True,
        metric_for_best_model='eval_auc',
        greater_is_better=True,
        fp16=True,
        num_train_epochs=epochs,              # total number of training epochs
        per_device_train_batch_size=batch_size,  # batch size per device during training
        per_device_eval_batch_size=batch_size,   # batch size for evaluation
        lr_scheduler_type='cosine',
        warmup_steps=500,                # number of warmup steps for learning rate scheduler
        weight_decay=0.01,               # strength of weight decay
        dataloader_num_workers=8,
        #logging_dir='./logs',            # directory for storing logs
        #logging_steps=10,
    )
        
    def fit(self, n_estimators=2):
        self.validation_preds = []
        self.test_preds = []
        for i in range(n_estimators):
            #bagging_trainset = self.trainingset.sample(frac=1, replace=True)
            bagging_trainset = resample(self.trainingset, replace=True, stratify=self.trainingset['labels'])
            bagging_trainset = self.get_dataset(bagging_trainset)
            train_encodings =  bagging_trainset.map(self.tokenize_function , batched = True)
            val_encodings = self.validationset.map(self.tokenize_function , batched = True)
            test_encodings = self.testset.map(self.tokenize_function , batched = True)
            
#             trainset = SpoilerDataset(train_encodings, bagging_trainset['class'].values)
#             valset = SpoilerDataset(val_encodings, self.validationset['class'].values)
#             testset = SpoilerDataset(test_encodings, self.testset['class'].values)
            
            trainer = Trainer(
                model=self.base_model, args=self.training_args, train_dataset=train_encodings['train'], eval_dataset=val_encodings['train'],
                compute_metrics=compute_metrics
            )
            
            trainer.train()
            
            val_preds = trainer.predict(val_encodings['train'])
            test_preds = trainer.predict(test_encodings['train'])
            self.validation_preds.append(val_preds)
            self.test_preds.append(test_preds)
            del bagging_trainset
            del train_encodings
            del val_encodings
            del test_encodings
            del trainer
            torch.cuda.empty_cache()
            print(f'{i + 1}. estimator is done....')
    def get_preds(self):
        return self.validation_preds, self.test_preds
    
    def get_metrics(self):
        #np.argmax((softmax(val[0].predictions, axis=1) + softmax(val[1].predictions, axis=1))/2, axis=1)
        val_preds = np.zeros_like(self.validation_preds[0].predictions)
        for val in self.validation_preds:
            val_preds += softmax(val.predictions, axis=1)
        
        val_logits = val_preds / len(self.validation_preds)
        val_preds = np.argmax(val_preds / len(self.validation_preds), axis=1)
        
        test_preds = np.zeros_like(self.test_preds[0].predictions)
        for test in self.test_preds:
            test_preds += softmax(test.predictions, axis=1)
        
        test_logits = test_preds / len(self.test_preds)
        test_preds = np.argmax(test_preds / len(self.test_preds), axis=1)
        precision = load_metric("precision")
        recall = load_metric("recall")
        f1 = load_metric("f1")
        acc = load_metric("accuracy")
        mcc = load_metric("matthews_correlation")
    
        val_precision = precision.compute(predictions=val_preds, average = "macro", references=self.validationset["train"]['labels'])["precision"]
        val_recall = recall.compute(predictions=val_preds, average = "macro", references=self.validationset["train"]['labels'])["recall"]
        val_f1 = f1.compute(predictions=val_preds, average = "macro", references=self.validationset["train"]['labels'])["f1"]
        val_acc = acc.compute(predictions=val_preds, references=self.validationset["train"]['labels'])["accuracy"]
        val_mcc = mcc.compute(predictions=val_preds, references=self.validationset["train"]['labels'])["matthews_correlation"]
        # val_auc = roc_auc_score(self.validationset["class"].values, softmax(val_logits, axis=1)[:, 1])
        
        test_precision = precision.compute(predictions=test_preds, average = "macro", references=self.testset["train"]['labels'])["precision"]
        test_recall = recall.compute(predictions=test_preds, average = "macro", references=self.testset["train"]['labels'])["recall"]
        test_f1 = f1.compute(predictions=test_preds, average = "macro", references=self.testset["train"]['labels'])["f1"]
        test_acc = acc.compute(predictions=test_preds, references=self.testset["train"]['labels'])["accuracy"]
        test_mcc = mcc.compute(predictions=test_preds, references=self.testset["train"]['labels'])["matthews_correlation"]
        # test_auc = roc_auc_score(self.testset["class"].values, softmax(test_logits, axis=1), multi_class='ovo', average='macro')

        return {"val_precision": val_precision, "val_recall": val_recall, "val_acc": val_acc, "val_mcc": val_mcc, "val_f1": val_f1}, {"test_precision": test_precision, "test_recall": test_recall, "test_acc": test_acc, "test_mcc": test_mcc, "test_f1": test_f1}
        

    def get_dataset(self, df): 
        df.to_csv('train.csv' , index = False)
        datast_train = load_dataset("csv", data_files="train.csv")
        return datast_train
        
    def tokenize_function(self , examples):
        return self.tokenizer(examples["review_sentences_join"], padding="max_length", truncation=True)

    def get_list_strs(self, df):
        lst_str = []
        for i in range(len(df)):
            str_ = df.values[i]
            lst_str.append(str_)
        return lst_str
    

In [7]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
def get_conf_mat(preds, truth):
    test_preds = np.zeros_like(preds[0].predictions)
    for test in preds:
        test_preds += softmax(test.predictions, axis=1)
    test_logits = test_preds / len(preds)
    test_preds = np.argmax(test_logits, axis=1)
    f1 = f1_score(truth, test_preds, average= "macro")
    return confusion_matrix(truth, test_preds), f1

In [8]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
def get_conf_mat(preds, truth):
    test_preds = np.zeros_like(preds[0].predictions)
    for test in preds:
        test_preds += softmax(test.predictions, axis=1)
    test_logits = test_preds / len(preds)
    test_preds = np.argmax(test_logits, axis=1)
    f1 = f1_score(truth, test_preds, average= "macro")
    return confusion_matrix(truth, test_preds), f1

In [9]:
df = pd.read_csv('../input/spoiler-non-context/train_sentences_ou.csv')
df_test = pd.read_csv('../input/spoiler-non-context/test_sentences.csv')

In [10]:
#rough

# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# def tokenize_function(examples):
#     return tokenizer(examples["review_sentences_join"], padding="max_length", truncation=True)
# tokenizer = BertTokenizer.from_pretrained("distilbert-base-uncased")



In [11]:
#rough end

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_val = train_test_split(df,
test_size=0.01, random_state=42, stratify = df['labels'])

In [13]:
X_train, X_res = train_test_split(X_train,
test_size=0.65, random_state=42, stratify = X_train['labels'])

In [14]:
from datasets import load_dataset

In [15]:
X_train.to_csv('train.csv' , index = False)
dataset_train = load_dataset("csv", data_files="train.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-b998c9f2b55ad723/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
X_val.to_csv('val.csv' , index = False)
dataset_val = load_dataset("csv", data_files="val.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-2bef7b3021a9229e/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
X_train

,review_sentences_join,labels
229170,"The Dresden Files series, by and large, is fun...",1
59557,The last 25 pages gave me whiplash!I'm still n...,0
288664,"Well, this might possibly be my favorite from ...",1
48761,"I finished reading this book in June.Yeah, I k...",0
263981,This has been sitting on my shelf for a while ...,1
...,...,...
275381,I started reading this with just a fair amount...,1
188189,Such a different read that is very hard to put...,0
25454,Good if you're really interested in the behind...,0
120028,A re-read but one I want to remember because I...,0


In [18]:

X_test, X_res_t = train_test_split(df_test,
test_size=0.6, random_state=42, stratify = df_test['labels'])
X_test.to_csv('test.csv' , index = False)
dataset_test = load_dataset("csv", data_files="test.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-34496591f2ce9b23/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
class_weights = (1 - (X_train['labels'].value_counts().sort_index() / len(X_train))).values
class_weights = torch.from_numpy(class_weights).float().to("cuda")
class_weights

tensor([0.3333, 0.6667], device='cuda:0')

In [20]:
from transformers import Trainer
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        outputs = model(**inputs)
        logits = outputs.get("logits")
        labels = inputs.get("labels")
        loss_func = nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_func(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [21]:
X_train

,review_sentences_join,labels
229170,"The Dresden Files series, by and large, is fun...",1
59557,The last 25 pages gave me whiplash!I'm still n...,0
288664,"Well, this might possibly be my favorite from ...",1
48761,"I finished reading this book in June.Yeah, I k...",0
263981,This has been sitting on my shelf for a while ...,1
...,...,...
275381,I started reading this with just a fair amount...,1
188189,Such a different read that is very hard to put...,0
25454,Good if you're really interested in the behind...,0
120028,A re-read but one I want to remember because I...,0


In [22]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100446 entries, 229170 to 239711
Data columns (total 2 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   review_sentences_join  100446 non-null  object
 1   labels                 100446 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.3+ MB


In [23]:
classifier = RandomTransformerClassifier(X_train, dataset_val , dataset_test, num_classes=2, 
                                         epochs=5, batch_size=8, max_sequence_length=450, model_name = "distilbert-base-uncased") 
                                         

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

In [24]:

classifier.fit(n_estimators=2)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-6f207a310ddef3bd/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

...

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: review_sentences_join, token_type_ids. If review_sentences_join, token_type_ids are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2899
  Batch size = 8


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Saving model checkpoint to ./results/checkpoint-62780
Configuration saved in ./results/checkpoint-62780/config.json
Model weights saved in ./results/checkpoint-62780/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results/checkpoint-12556 (score: 0.864565426251474).
The following columns in the test set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: review_sentences_join, token_type_ids. If review_sentences_join, token_type_ids are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 2899
  Batch size = 8


The following columns in the test set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: review_sentences_join, token_type_ids. If review_sentences_join, token_type_ids are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 137803
  Batch size = 8


2. estimator is done....


In [25]:
val, test = classifier.get_metrics()


In [26]:
val 

{'val_precision': 0.780295810749668,
 'val_recall': 0.7873083172403894,
 'val_acc': 0.8051052086926527,
 'val_mcc': 0.5675608080761665,
 'val_f1': 0.7834892787910872}

In [27]:
test

{'test_precision': 0.5973664777669612,
 'test_recall': 0.7585832471158552,
 'test_acc': 0.8216874814046138,
 'test_mcc': 0.31734738052307626,
 'test_f1': 0.615312452381203}

In [28]:
cm, f1 = get_conf_mat(classifier.get_preds()[1], dataset_val['train']["labels"])

ValueError: Found input variables with inconsistent numbers of samples: [2899, 137803]

In [ ]:
cm


In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import itertools


def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          f1 = None,
                          cmap=None,
                          normalize=True):
    

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}; f1 = {:0.4f}'.format(accuracy, misclass,f1))
    plt.show()

In [ ]:
target_names = ["nospoiler", "spoiler"]

In [ ]:
plot_confusion_matrix(cm,
                      target_names,
                      title='Confusion matrix',
                      f1 = f1,
                      cmap=None,
                      normalize=False)